You work as a product analyst for a delivery company. The service is available in both iOS and Android apps. You set up analytics in AppMetrica, and at the end of the quarter, the marketing manager asked you to analyze user behavior and evaluate the effectiveness of their acquisition channels.

You downloaded data from AppMetrica for the period from January 1 to March 31, 2020, only for users registered after January 1, 2020.

Data Description

- date – event date

- event

    app_install – app installed
    app_start – app opened
    register 
    search – visiting the product search page (catalog)
    open_item
    choose_item – add item to cart
    tap_basket – go to cart
    purchase – purchase approval

- gender

- os_name – user platform

- city

- device_id

- urm_source – the channel the user came from

    yandex-direct
    google_ads
    vk_ads
    instagram_ads
    facebook_ads
    referal
    If there is ‘-’, then the channel is not defined or this is a direct download of the application or a visit not from an advertisement  

- purchase_sum

Please note:

- The data includes unique user actions for each day

- You can skip the application installation stage if it was installed earlier

- You can skip the registration stage if the user was already logged in at the time of the session. However, unregistered users cannot make a purchase.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('KC_case_data .csv', parse_dates=['date'])

In [3]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2747968 entries, 0 to 2747967
Data columns (total 8 columns):
 #   Column        Dtype         
---  ------        -----         
 0   date          datetime64[ns]
 1   event         object        
 2   purchase_sum  float64       
 3   os_name       object        
 4   device_id     int64         
 5   gender        object        
 6   city          object        
 7   utm_source    object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 167.7+ MB


**Calculate the MAU for February**

In [5]:
df['month']=df.date.dt.month_name() #для удобства сделаем переменную с названием месяца

In [6]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,January
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,January
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,January
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,January
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,January


In [7]:
MAU=df.query("month=='February'").device_id.nunique()

In [8]:
print(f'Answer: MAU for February is {MAU}')

Answer: MAU for February is 75032


**Calculate number of installs in January*

In [9]:
installs = df.query("month=='January' & event == 'app_install'").device_id.count()

In [10]:
print(f'Answer: There were {installs} installs in January')

Answer: There were 80297 installs in January


**Assign users to cohorts by the day of app installation and calculate the conversion from installation to purchase for them within 7 days. For which cohort was the conversion the highest?**

Note: we consider a user converted if no more than 7 days have passed from the moment of installation to the first purchase.

In [11]:
#Let's create a dataframe with install dates
install_date = df.query("event == 'app_install'")[['device_id', 'date']].rename(columns={'date': 'install_date'})

In [12]:
install_date.head()

,device_id,install_date
2515061,4921563,2020-01-01
2515062,1311583,2020-01-01
2515063,23006325,2020-01-01
2515064,5645652,2020-01-01
2515065,8798567,2020-01-01


In [13]:
#Let's lef join to the original dataframe
full_data = pd.merge(df, install_date, on='device_id', how='left')

In [14]:
full_data.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month,install_date
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,January,2020-01-01
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,January,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,January,NaT
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,January,2020-01-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,January,2020-01-01


In [15]:
#let's do the same for the purchase date
purchase_date=df.query("event=='purchase'")[['device_id', 'date']].rename(columns={'date': 'purchase_date'})

In [16]:
full_data = pd.merge(full_data, purchase_date, on='device_id', how='left')

In [17]:
#now we have the data we need to calculate the time difference between installation and purchase
full_data.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month,install_date,purchase_date
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,January,2020-01-01,2020-01-07
1,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,January,2020-01-01,2020-01-09
2,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,January,2020-01-01,2020-01-01
3,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,January,2020-01-01,2020-01-11
4,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,January,2020-01-01,2020-01-16


In [18]:
full_data['timediff'] = (full_data['purchase_date'] - full_data['install_date']).dt.days

In [19]:
full_data.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month,install_date,purchase_date,timediff
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,January,2020-01-01,2020-01-07,6.0
1,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,January,2020-01-01,2020-01-09,8.0
2,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,January,2020-01-01,2020-01-01,0.0
3,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,January,2020-01-01,2020-01-11,10.0
4,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,January,2020-01-01,2020-01-16,15.0


In [20]:
#Let's add the users who made a purchase during the week to a separate data frame 
seven_days_purchase = full_data.query("timediff<=7 & event=='purchase'")

In [21]:
#calculating conversion based on the dataframe with installations
#to find out the ids for purchases, let's calculate the number of ids from the dataframe with purchases
cohort_conversion = install_date.groupby('install_date').agg(
    total_installs=('device_id', 'nunique'),
    total_purchases=('device_id', lambda x: x.isin(seven_days_purchase['device_id']).sum())
).reset_index()

In [22]:
cohort_conversion['conversion_rate'] = cohort_conversion['total_purchases'] / cohort_conversion['total_installs']

In [23]:
cohort_sorted = cohort_conversion.sort_values('conversion_rate',ascending = False)

In [24]:
print(f'Answer: the highest conversion was on {cohort_sorted.install_date[0].date()}')

Answer: the highest conversion was on 2020-01-01


**Which paid marketing channel generated the highest number of new users?** 

In [25]:
channel = df.groupby('utm_source', as_index = False).device_id.nunique()\
.sort_values('device_id', ascending = False).reset_index(drop= True)

In [26]:
channel = channel.query("utm_source != '-'")

In [27]:
channel

,utm_source,device_id
1,yandex-direct,40712
2,google_ads,38096
3,vk_ads,34086
4,instagram_ads,31048
5,facebook_ads,25959
6,referal,15926


In [28]:
print(f'Answer: The largest number of users came from {channel.utm_source[1]}')

Answer: The largest number of users came from yandex-direct


**Users who came from which channel have a higher median first check? (only users' first purchases are taken into account)**

In [29]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,January
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,January
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,January
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,January
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,January


In [30]:
median_check = df.query("event == 'purchase'")[['device_id', 'date','purchase_sum','utm_source']]\
.groupby('device_id', as_index = False)\
.agg(first_date = ('date', 'min'), first_check = ('purchase_sum', 'first'), utm_source = ('utm_source','first'))\
.groupby('utm_source', as_index = False).agg(median_check=('first_check','median'))\
.sort_values('median_check', ascending = False).reset_index(drop=True)
median_check = median_check.query("utm_source != '-'")

In [31]:
median_check

,utm_source,median_check
1,referal,395.5
2,instagram_ads,393.5
3,vk_ads,393.0
4,yandex-direct,392.5
5,google_ads,390.5
6,facebook_ads,389.0


In [32]:
print(f'Answer: Users who came from the channel {median_check.utm_source[1]} have the highest median check')

Answer: Users who came from the channel referal have the highest median check


**Which channels had the lowest first purchase conversion rate?**

Hint:

- CR = number of the first time purchases/number of the first time app openers

In [33]:
#Let's select the users who opened the app and made a purchase
all_users=full_data.query("event=='app_start'")
purchases=full_data.query("event=='purchase'")


In [34]:
#Let's create a dataframe with the first purchases
first_purchases = purchases.groupby('device_id', as_index = False).apply(lambda x: x.sort_values('date'))

first_purchases = first_purchases.groupby('device_id').head(1)

In [35]:
#Let's do the same for the first time the users opened the application
first_opened = all_users.groupby('device_id', as_index = False).apply(lambda x: x.sort_values('date'))

first_opened = all_users.groupby('device_id').head(1)


In [36]:
#Let's calculate the number of unique users who made their first purchase
first_purchase_users = first_purchases.groupby('utm_source', as_index = False).device_id.nunique().rename(columns = {'device_id':'purchases'})

#And opened the app for the first time
first_opened_users = first_opened.groupby('utm_source', as_index = False).device_id.nunique().rename(columns = {'device_id':'opened'})



In [37]:
#Let's calculate the CR
CR = pd.merge(first_opened_users, first_purchase_users, on = 'utm_source', how = 'left')
CR['CR'] = CR['purchases'] / CR['opened']
CR = CR.query("utm_source != '-'").sort_values('CR').reset_index(drop = True)
CR

,utm_source,opened,purchases,CR
0,yandex-direct,34441,10936,0.317529
1,google_ads,31437,10167,0.323409
2,instagram_ads,24818,9820,0.395681
3,vk_ads,27905,11460,0.410679
4,facebook_ads,18844,7903,0.419391
5,referal,11983,5803,0.484269


In [38]:
print(f'Answer: The lowest conversion rate to first purchase is for users who came from {CR.utm_source[0]}')

Answer: The lowest conversion rate to first purchase is for users who came from yandex-direct


**Advertising cost data**:

Yandex – 10,491,707 rubles.

Google – 10,534,878 rubles.

Facebook – 8,590,498 rubles.

Instagram – 8,561,626 rubles.

VK – 9,553,531 rubles.

Referral program costs: if a user brings a friend and the latter makes the first purchase, both will receive 100 rubles.

Which paid acquisition channel (among advertising) has the highest ROMI?**

In [39]:
#ROMI - (gross profit-marketing costs) / marketing costs
#Gross profit  — total revenue minus costs of goods.
#We don't have any data on costs of goods, so we'll calculate the ROMI based on total revenue
#To begin with, let's calculate the number of users and revenue for all users
ROMI = full_data.groupby('utm_source', as_index = False).agg(revenue = ('purchase_sum','sum'), clients = ('device_id','nunique'))

In [40]:
ROMI

,utm_source,revenue,clients
0,-,108901736.5,52273
1,facebook_ads,60668237.5,25959
2,google_ads,51002137.0,38096
3,instagram_ads,68598287.5,31048
4,referal,45357215.5,15926
5,vk_ads,73764243.0,34086
6,yandex-direct,58175485.5,40712


In [41]:
#Let's write a function that will fill in the costs based on the source

def costs (x):
    if x['utm_source'] == 'facebook_ads':
        return 8590498
    elif x['utm_source'] == 'google_ads':
        return 10534878
    elif x['utm_source'] == 'instagram_ads':
        return 8561626
    elif x['utm_source'] == 'vk_ads':
        return 9553531
    elif x['utm_source'] == 'yandex-direct':
        return 10491707
    elif x['utm_source'] == 'referal':
        return x['clients'] * 200

In [42]:
ROMI['advertising_costs'] = ROMI.apply(costs, axis = 1)

In [43]:
#Let's calculate ROMI
ROMI['ROMI'] = (ROMI['revenue'] - ROMI['advertising_costs']) - ROMI['advertising_costs']

In [44]:
ROMI = ROMI.query("utm_source != '-'").sort_values('ROMI', ascending = False).reset_index(drop = True)

In [45]:
ROMI

,utm_source,revenue,clients,advertising_costs,ROMI
0,vk_ads,73764243.0,34086,9553531.0,54657181.0
1,instagram_ads,68598287.5,31048,8561626.0,51475035.5
2,facebook_ads,60668237.5,25959,8590498.0,43487241.5
3,referal,45357215.5,15926,3185200.0,38986815.5
4,yandex-direct,58175485.5,40712,10491707.0,37192071.5
5,google_ads,51002137.0,38096,10534878.0,29932381.0


In [46]:
print(f'Answer: The lowest ROMI is for {ROMI.utm_source[0]}')

Answer: The lowest ROMI is for vk_ads
